# Lyrics Generation Test
Here will test the effeciacy of our models. To see if the network learns to create more novel lyrics

Each model will be fed a seed line of text to base 100 characters of prediction on.

To judge the progress of the model over epoch we will test the 01, 05, 10, 15, and 20 epoch weights.

In [1]:
# Imports
import numpy as np
import sys
import os
import random
from collections import OrderedDict

from keras.models import load_model

Using TensorFlow backend.


## Model Creation
For each sequence length a model is constructed for each epoch

In [2]:
# Function to iterate through a directory of model weights
def model_fabricator(char, model_dict):
    # Creating a list of all weights to create models from
    # Sorting list for easier lyric comparision based on model weight
    for weight in sorted(os.listdir(path=f'../model-building/model-weights/{char}-char-model-weights')):
        temp_model = load_model(f'../model-building/models/{char}-char-seq-shanty-writer.h5')
        temp_model.load_weights(f'../model-building/model-weights/{char}-char-model-weights/{weight}')
        # Adding model to models_char dict
        # Key is epoch number found in weight string
        model_dict[int(weight[-14:-12])] = temp_model
    return model_dict

In [3]:
# Creating empty dicts for each character sequence length
models_25  = OrderedDict()
models_50  = OrderedDict()
models_100 = OrderedDict()

In [4]:
# Populating each models list
model_fabricator(25, models_25)
model_fabricator(50, models_50)
model_fabricator(100, models_100);

## Corpus Data and Dictionaries
In order to convert the outputs from the model we will need translation dictionaries

Loading in the lyrics corpus will let us create these dictionaries

In [5]:
# Loading in Shanties lyrics corpus
shanties = open('../../../data/shanties_all.txt', encoding='utf-8').read()

# Creating a list of all unique characters and variable of total number of uniques
chars_list = sorted(list(set(shanties)))
n_chars = len(chars_list)

In [6]:
# Creating a dictionary to map each unique character to a number
chars_to_ints = dict((c, i) for i, c in enumerate(chars_list))

# Creating a dictionary to return numbers to characters
ints_to_chars = dict((i, c) for  i, c in enumerate(chars_list))

## Lyrics Writing
Using a function outputing the input string and generated lyrics by epoch to compare the different model sequences

In [7]:
def lyrics_writer(seed_string, models_dict, model_char_len ,n_predictions):
    # Converting seed string into a list of numbers
    for epoch, model in models_dict.items():
        # Converting seed string into a list of numbers
        # Truncating input string by the character sequence length of the model inputed
        seq_pattern = [chars_to_ints[char] for char in seed_string[:model_char_len].lower()]
        # Creating empty string to append results too
        output_string = ''
        # Each iteration in loop will predict one character
        for i in range(n_predictions):
            # Creating x data to predict off. Reshaping and normalizing input list
            x = np.reshape(seq_pattern, (1, len(seq_pattern), 1))
            x = x / float(n_chars)
            # Creating predictions
            preds = model.predict(x, verbose=0)
            # Ordering predictions by most likely character
            index = np.argmax(preds)
            # Converting result to character
            result = ints_to_chars[index]
            # Reading in current pattern
            seq_in = [ints_to_chars[value] for value in seq_pattern]
            # Adding result to input list
            seq_pattern.append(index)
            # Shifting pattern over one index for next pass through loop
            seq_pattern = seq_pattern[1:len(seq_pattern)]
            # Appending the result seed_string to print as final output
            output_string = output_string + result
            
        print()
        print(f'Epoch {epoch}:')
        print(f'{seed_string[:model_char_len]}{output_string}')
        print()

In [8]:
input_string = 'Washington DC is the best campus of General Assembly by far and away no other campus can possibly compare'
len(input_string)

105

## 25 Character Sequence Model

In [9]:
lyrics_writer(input_string, models_25, 25, 100)


Epoch 1:
Washington DC is the best and she sail and she sail and she sail and she sail and she sail and she sail and she sail and she 


Epoch 2:
Washington DC is the best she shan she shan she shan she shan she shan she shan she shan she shan she shan she shan she shan 


Epoch 3:
Washington DC is the best of the sailor lad the shall be the shanty and the shall be the shanty and the shall be the shanty a


Epoch 4:
Washington DC is the best and the sailor s the sailor sound the sailor s the sailor sound the sailor s the sailor sound the s


Epoch 5:
Washington DC is the best and the sailor love the sailor love the sailor love the sailor love the sailor love the sailor love


Epoch 6:
Washington DC is the best of the sea and the soack let the boat and the soack let the boat and the soack let the boat and the


Epoch 7:
Washington DC is the best and the sailor loves the boat and the sailor loves the boat and the sailor loves the boat and the s


Epoch 8:
Washington DC is the best of th

## 50 Character Sequence Model

In [10]:
lyrics_writer(input_string, models_50, 50, 100)


Epoch 1:
Washington DC is the best campus of General Assembn the sail the sailor sail the sailor sail the sailor sail the sailor sail the sailor sail the sailo


Epoch 2:
Washington DC is the best campus of General Assembe the sailor s bootier she s a sailor s bootier the sailor s bootier she s a sailor s bootier the sa


Epoch 3:
Washington DC is the best campus of General Assembe the sailor s io the sailor s io the sailor s io the sailor s io the sailor s io the sailor s io th


Epoch 4:
Washington DC is the best campus of General Assembe the ship was a sailor s a sailor s a sailor s a sailor s a sailor s a sailor s a sailor s a sailor


Epoch 5:
Washington DC is the best campus of General Assembe the sailor love we ll roar and the sailor love we ll roar and the sailor love we ll roar and the s


Epoch 6:
Washington DC is the best campus of General Assemb the sailor s land the sailor love the sailor s land the sailor love the sailor s land the sailor lo


Epoch 7:
Washington DC is t

## 100 Character Sequence Model

In [11]:
lyrics_writer(input_string, models_100, 100, 100)


Epoch 1:
Washington DC is the best campus of General Assembly by far and away no other campus can possibly coow the sailor she sailor she sailor she sailor she sailor she sailor she sailor she sailor she sailo


Epoch 2:
Washington DC is the best campus of General Assembly by far and away no other campus can possibly come and the ship she sailor s bous the brave and the ship was a ship she sailor say and the ship was 


Epoch 3:
Washington DC is the best campus of General Assembly by far and away no other campus can possibly come and the sailor s the sailor s the sailor s the sailor s the sailor s the sailor s the sailor s th


Epoch 4:
Washington DC is the best campus of General Assembly by far and away no other campus can possibly come all you re the sailor lad a ship so sea the sailor lad a ship so sea the sailor lad a ship so sea


Epoch 5:
Washington DC is the best campus of General Assembly by far and away no other campus can possibly come and the sea and the sailor lad so sea t

---
## 50 Character Sequence Analysis
We can see several noticeable things about the lyrics written:

**Epoch 1**
- Even from the first epoch we see fully spelled words
- The first character result for "Assemb" is "n". This doesn't make much grammatical sense.

**Epochs 2 - 4**
- The model begins to come up with novel words such as "bootier"
- Has replaced the "n" on "Assemb" with an "e".

**Epochs 5 - 9**
- Epoch 6 drops the "e" from "Assemb" but it returns with the later epochs
- We see a bit more variety in terms of words but "sailor" is still very prevalent

**Epochs 10 - 19**
- "Assembe" is now "Assember". While still a nonsense word it makes some lyrical sense.
- While the pattern of words is still generally short we see a bit more variety in word choice by epoch

**Epoch 20**
- The final epoch has less variety but seems to make the most sense as a single lyrics
 - "the sailor loves the sea and the ship"

Over all the model created what could be interpreted as lyrics for a line for a song.

## Next Steps

It seems the model may have been over fit as every output was a repetition of a few words.

To minimize repetition we will try to following
- Run the same model but with different sequence and step lengths
- Run a less performant model with variable sequence and step lengths
- Process the corpus differently:
 - Analyze the document by words rather then characters
 - One hot encoding
- Research alternate ways bring in variability into text prediction